In [1]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [2]:
ca = census.CensusData()
tmp_df = ca.get_data_for_city("norfolk", years=(2019, 2024))

<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>


In [ ]:
pgpw = os.getenv("POSTGRES_PASSWORD")
server, engine = ca.connect_to_postgres()
ca.upload_city_data_to_postgres(tmp_df)

In [ ]:
co = climate.ClimateAgent()
station_id = co.get_station_id("Norfolk, Virginia")
climate_df = co.get_data_for_station_id(station_id=station_id, start_date="2022-01-01", end_date="2022-12-31")

In [ ]:
# ca = census.CensusData()

# pprint(ca.access_economic_data("Norfolk", [2022]))

In [ ]:
from html.parser import HTMLParser
#census_url = "https://api.census.gov/data"
year = "2023"
dataset = "acs"
url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


"""
    # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

    # ones worth using:

    # economic
    DP03_0093E = estimate for earnings for first year male workers 
    DP03_0094E = estimate for earnings for first year female workers
    DP03_0119PE = estimated percentage of people below the poverty line
    DP04_0001E = estimated housing units
    DP02_0002E = estimated number of married housing units
    DP02_0016E = estimated average household size

    DP03_0052E: estimated total households less than $10,000 income
    DP03_0053E: estimated total households between $10-15k
    DP03_0054E: estimated total households between $15-25k
    DP03_0055E: 25-35k
    DP03_0056E: 35-45k
    DP03_0057E: 50-75k
    DP03_0058E: 75-100k
    DP03_0059E: 100-150k
    DP03_0060E: 150-200k
    DP03_0061E: > 200k

    # relationships
    DP02_0026E = men over 15 y/o who have not married
    DP02_0028E = men over 15 y/o who married, but separated
    DP02_0030E = men over 15 y/o who are divorced
    
    # education (currently)
    DP02_0054E = estimated population of kids in nursery school/preschool
    DP02_0055E = estimated population of kids in kindergarten
    DP02_0056E = estimated population of kids in grades 1-8
    DP02_0057E = estimated population of kids in grades 9-12
    DP02_0058E = estimated population of kids in college/grad school

    # education attainment
    DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
    DP02_0061PE = estimated population with some high school education
    DP02_0062PE = estimated population with high school diploma
    DP02_0063PE = estimated population with some college, no diploma
    DP02_0064PE = estimated population with associates degrees
    DP02_0065PE  = estimated population with bachelor's degree
    DP02_0066PE = estimated population with graduate degrees

    # unemployement rate
    DP03_0009PE

    # industry
    DP03_0033PE: agriculture, forestry, fishing, hunting, mining
    DP03_0034PE: construction
    DP03_0035PE: manufacturing
    DP03_0036PE: wholesale trade
    DP03_0037PE: retail trade
    DP03_0038PE: transportation, warehousing, utilities
    DP03_0039PE: information
    DP03_0040PE: finance, insurance, real estate, rental, leasing
    DP03_0041PE: professional, scientific, administrative, waste management
    DP03_0042PE: educational services/health care/ social assistance
    DP03_0043PE: arts, entertainment, recreation, accommodation, food services
    DP03_0045PE: public administration

"""
variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
print(url)
params = {"key": os.getenv("CENSUS_KEY"),
           "for": "county:*", 
           "get": variables,
           "in": "state:51"
          }
r = requests.get(url, params=params)
values = list(json.loads(r.text))

df = pd.DataFrame(values[1:], columns=values[0])
# df = df.rename(columns={
#     "DP02_0060PE":"<9th",
#     "DP02_0061PE": "some high school",
#     "DP02_0062PE": "high school",
#     "DP02_0063PE": "some college",
#     "DP02_0064PE": "associates",
#     "DP02_0065PE": "bachelors",
#     "DP02_0066PE": "graduate"
# })
df

https://api.census.gov/data/2023/acs/acs1/profile


,NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,state,county
0,"Albemarle County, Virginia",2.0,4.4,14.5,11.8,5.4,29.0,32.8,51,003
1,"Arlington County, Virginia",2.6,1.6,7.9,7.0,2.9,36.8,41.2,51,013
2,"Augusta County, Virginia",3.3,5.5,41.4,17.3,7.8,16.3,8.4,51,015
3,"Bedford County, Virginia",1.9,6.7,30.1,19.9,9.0,19.6,12.9,51,019
4,"Chesterfield County, Virginia",3.1,3.6,20.3,19.2,9.2,29.0,15.6,51,041
5,"Fairfax County, Virginia",4.4,3.5,11.3,11.5,5.5,31.2,32.7,51,059
6,"Fauquier County, Virginia",2.4,5.4,24.2,20.1,6.5,27.3,14.0,51,061
7,"Frederick County, Virginia",3.8,5.5,29.6,20.3,7.4,20.7,12.7,51,069
8,"Hanover County, Virginia",1.7,4.4,24.6,17.6,7.9,26.2,17.6,51,085
9,"Henrico County, Virginia",3.9,3.8,21.6,17.3,7.5,26.5,19.5,51,087


In [ ]:
df['NAME'] = df['NAME'].str.lower()
df['NAME'] = df['NAME'].str.replace(" ", "")

In [ ]:
df[df['NAME'].str.contains("norfolk")]